In [ ]:
from kyle_tools import rc_dict

rc_dict['legend.loc'] = 'upper right'
rc_dict['legend.fontsize'] = 12
for k,v in rc_dict.items():
    plt.rcParams[k] = v



In [ ]:
fl_diff

In [ ]:
#THIS IS FOR IMPORTING RATE BASED EP MEASUREMENTS, not regularized ones


fl_diff = kt.file_list('data/free_diff/')
#import data
#flst = kt.file_list('24_03_13/')
#flst = kt.file_list('24_03_14/')
fl_diff = kt.file_list('data/free_diff/')

fl_diff.sort()

data_list = [kt.open_json(item) for item in fl_diff if 'steps248' in item ]
#data_list = [kt.open_json(item) for item in flst ][1:2]
skip=1


data = [ d['data'] for d in data_list]

print([d.shape for d in data])

dt = data_list[0]['params']['sim'][-1]
#proc_dt = data_list[0]['proc_params']['sim'][-1]
steps = data_list[0]['steps']
#num_iter = len(all_estimates)
params = data_list[0]['params']
times =  data_list[0]['t']

rate_EP = []
rate_N = [i['params']['sim'][0] for i in data_list ]
rate_E_list = [ i['num_ests'] for i in data_list ]




for item in rate_E_list:
    try: valid_ests = np.intersect1d(valid_ests, item)
    except: valid_ests = item
valid_ests = np.array([6,8, 10])


EP_skip =1

theo = data_list[0]['theo'][1]
print(theo[-1])
rate_theo = dt * np.cumsum(theo)[steps[-1]]
print(rate_theo)

rate_E = []
for vals, estimates  in zip(data, rate_E_list):
    msk = np.any([estimates == ve for ve in valid_ests], axis=0)
    all_e = vals[:,msk,:]
    ssteps = steps[::EP_skip][:-1]
    rate_steps = len(ssteps)
    rate_E.append(np.array(estimates)[msk])
    EP = np.cumsum((np.diff(times[steps[::EP_skip]]))*(all_e[...,::EP_skip][...,:-1]), axis=-1)[...,-1]
    #EP[EP>2.5] = np.nan
    EP_M = np.nanmean(EP, axis=0)
    EP_S = 3*np.nanstd(EP, axis=0)/(np.sqrt(EP.shape[0])) 
    
    rate_EP.append( [EP_M, EP_S] )

rate_EP, rate_N, rate_E =[ np.array(item) for item in [rate_EP, rate_N, rate_E ]]

#all_estimates = np.vstack(data)/skip**2

#all_e = all_estimates

#means = all_e.mean(axis=0)
#stds = all_e.std(axis=0)



In [ ]:
fl_diff

In [ ]:
rate_steps

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color'][4:]
color_dict = {k:v for k,v in zip(valid_ests,colors)}

fig, ax = plt.subplots()

for E in valid_ests:
    msk = np.where(rate_E ==E)
    temp_EP = rate_EP[msk[0], :, msk[1]]
    temp_N = [np.random.uniform(.95,1.05)*i for i in rate_N]
    ax.errorbar( temp_N, (temp_EP[:,0]-rate_theo)/rate_theo, yerr=temp_EP[:,1], color=color_dict[E], alpha=1, markersize=4, linewidth=1.5, marker='D', label=f'{E}')

ax.set_yscale('symlog')
ax.set_yticks([-.05,.01, .1, .5])
ax.axhline(0, c='k')

ax.set_ylim(-.01,.01)
fig.legend()

In [ ]:
dr = './data/oneshot/steps_250/'
#dr = './data/oneshot/steps_250/temp_off/'
#dr = './data/oneshot/'
#dr = './data/oneshot/steps_50/'

#dr = './data/oneshot/'

fl = kt.file_list(directory = dr)

fl.sort()


In [ ]:
fl

In [ ]:
d = [kt.open_json(f) for f in fl]

In [ ]:
from weight_discovery import add_regularizer

for i in range(len(d)):

    def alpha(data, N = d[i]['params']['sim'][0]):
        return 1/N**2

    #print(alpha(d[i]['data']))
    add_regularizer(d[i], 1, alpha_gen = alpha, verbose=True )

In [ ]:
data =[]
params = []
for dct in d:
    temp_list = []
    temp_list.extend(dct['num_est'])
    temp_list.append(dct['proc_params']['sim'][1])
    temp_list.append(dct['params']['sim'][0])
    if not temp_list in params:
        params.append(temp_list)

        data_key = 'reg_data'

        data.append(dct[data_key])
        #data.append(dct['data'])
    else:
        #print('dupe',temp_list)
        loc = np.where((np.array(params)==temp_list).all(axis=-1))
        #print(loc)
        data[ loc[0][0] ] = np.append(data[ loc[0][0] ], dct[data_key])


params = np.array(params)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(20,12), width_ratios=[.8,2])
#fig, ax = plt.subplots(1,2, figsize=(8,4), width_ratios=[.8,2])
ax = ax[::-1]
axR = 0
axL = 1

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
theo = d[0]['theo']
#theo = rate_theo
rate_theo = theo

C = list(set( [ tuple(i) for i in params[:,:-1]])  )
C.sort()
C = np.array(C)

elist = list(set(params[:,:-2].prod(axis=-1)))
elist.sort()


lbls=[]
for val, param in zip(data, params):
    
    np.random.shuffle(val)
    val = val[:]
    
    '''
    L1 = len(val)
    m, s = np.mean(val), 4*np.std(val)
    LL, UL = m-s, m+s
    #LL, UL = 1.2, 2.1
    val = val[val<UL]
    val = val[val>LL]
    L2 = len(val)
    while L1-L2 > 0:
        print(L1-L2, param)
        L1 = len(val)
        m, s = np.mean(val), 4*np.std(val)
        LL, UL = m-s, m+s
        val = val[val<UL]
        val = val[val>LL]
        L2 = len(val)
    '''

    
    color_code = param[:-1]
    c = colors[ np.where( (C==color_code).all(axis=-1)) [0][0] ]
    #m, s = np.mean(data)/theo-1, (3*np.std(data)/np.sqrt(len(data)-1))/theo
    m, s = np.mean(val), (3*np.std(val)/np.sqrt(len(val)-1))
    #print(m,s)
    if abs((m-theo)/theo) < 100:

        steps = param[-2]
        #lbl = f"oneshot:{list(param[:2])}$\\times${param[2]}"
        lbl = f"E:{param[:3]} S:{steps}_t:{len(val)}_N{param[-1]/10_000}"
        if lbl in lbls:
            lbl = None

        offset = 1+.025 * ( -(len(elist)-1)/2 + (np.where(param[:-2].prod() == elist)[0][0]))
        ax[axR].errorbar(param[-1]*offset, (m-theo)/theo, marker='o', yerr=s/theo, label=lbl, color=c)
        
        #ax[1].errorbar(param[-1]*np.random.uniform(.9,1.1), (m-theo)/theo, marker='o', yerr=s/theo, markersize=4,linewidth=.8, color=c)
        ax[axL].errorbar(param[-1]*offset, (m-theo)/theo, marker='o', yerr=s/theo, markersize=4,linewidth=.8, color=c)

        lbls.append(lbl)



for E in valid_ests:
    msk = np.where(rate_E ==E)
    temp_EP = rate_EP[msk[0], :, msk[1]]
    temp_N = [np.random.uniform(.9,1.1)*i for i in rate_N]
    if E == 10:
        temp_EP = temp_EP[1:]
        temp_N = temp_N[1:]
        temp_EP[0,1]=0
    lbl = f'rate {[E,E]}$\\times${rate_steps+3}'


    ax[axL].errorbar( temp_N, (temp_EP[:,0]-rate_theo)/rate_theo, yerr=temp_EP[:,1]/rate_theo, color=color_dict[E], alpha=1, markersize=3, linestyle='--', linewidth=2, marker='D', label=lbl)
    #ax[1].errorbar( temp_N, temp_EP[:,0], yerr=temp_EP[:,1], color=color_dict[E], alpha=1, markersize=6, linestyle='--', linewidth=2, marker='D', label=lbl)

    plot_err = [ item[1]/rate_theo if (item[0]-rate_theo)/rate_theo<.05 else 0 for item in temp_EP[:] ]
    ax[axR].errorbar( temp_N[:], (temp_EP[:,0]-rate_theo)/rate_theo, yerr=plot_err, color=color_dict[E], alpha=.5, markersize=6, linestyle='--', linewidth=1, marker='D')

    #ax[0].plot( temp_N[:], (temp_EP[:,0]-rate_theo)/rate_theo, color=color_dict[E], alpha=.5, markersize=4, linestyle='--', linewidth=1, marker='D')



ax[axL].axhline(0, c='k')


ax[axL].set_yscale('symlog')
ax[1].set_yticks([-1,.1,1,100,1000])
ax[axL].set_ylim(-.5,2.5)

#print(d[0]['theo']);

ax[axR].set_xlabel('number of paths')
ax[axL].set_xlabel('number of paths')

ax[axL].set_ylabel('relative error')

ax[axR].set_xscale('log')
ax[axL].set_xscale('log')

#ax[0].set_title('oneshot $\Sigma$ estimation')
#ax[1].set_title('comparison to rate $\Sigma$ estimation')
ax[axR].set_ylim(-.06, .06);
#ax.set_ylim(1.6, 2)
#ax.set_ylim(-80, 50);
#ax.axhline(theo, color='k', label='theo')
ax[axR].set_xlim(5_000, 550_000)
ax[axL].set_xlim(5_000,1_500_000)

ax[axR].axhline(0, color='k')
ax[axL].axhline(0, color='k')

ax[axL].axhspan(-.05, .05, xmax=.83, facecolor='k', alpha=0.2)
ax[axR].axhspan(-.05, .05, facecolor='k', alpha=0.2)

fig.legend()
fig.tight_layout()

In [ ]:
fig.savefig('reg_test_250_p1_j50.pdf')

In [ ]:
data_list[0]['data'].shape

In [ ]:
data_list[0]['reg_data']

In [ ]:
#importing  regularized rate based estimation
fl_diff = kt.file_list('data/free_diff/mats/')
fl_diff.sort()
data_list = [kt.open_json(item) for item in fl_diff]

skip=1

data = [ d['reg_data'] for d in data_list]

dt = data_list[0]['params']['sim'][-1]
#proc_dt = data_list[0]['proc_params']['sim'][-1]
steps = data_list[0]['steps']
#num_iter = len(all_estimates)
params = data_list[0]['params']
times =  data_list[0]['t']

EP_skip=1

rate_EP = []
rate_N = [i['num_path'] for i in data_list ]

theo = data_list[0]['theo'][1]
print(theo[-1])
rate_theo = dt * np.cumsum(theo)[steps[-1]]
print(rate_theo)

rate_E = []

for file  in data_list:
    temp_rate_EP = []
    estimates = file['num_ests'][0]
    print(estimates)
    rate_E.append(estimates)
    
    for vals in file['reg_data']:
        all_e = vals[:,0,:]
        ssteps = steps[::EP_skip][:-1]
        rate_steps = len(ssteps)
        EP = np.cumsum((np.diff(times[steps[::EP_skip]]))*(all_e[...,::EP_skip][...,:-1]), axis=-1)[...,-1]
        EP_M = np.nanmean(EP, axis=0)
        EP_S = 3*np.nanstd(EP, axis=0)/(np.sqrt(EP.shape[0])) 
    
        temp_rate_EP.append( [EP_M, EP_S] )

    rate_EP.append(np.array(temp_rate_EP))


In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color'][4:]
color_dict = {k:v for k,v in zip(rate_E,colors)}

fig, ax = plt.subplots()

for E, N, EP in zip(rate_E, rate_N, rate_EP):
    temp_N = [np.random.uniform(.95,1.05)*i for i in N]
    ax.errorbar(temp_N, (EP[:,0]-rate_theo)/rate_theo, yerr=EP[:,1], color=color_dict[E], alpha=1, markersize=4, linewidth=1.5, marker='D', label=f'{E}')

#ax.set_yscale('symlog')
ax.set_yticks([-.05,.01, .1, .5])
ax.axhline(0, c='k')

ax.set_ylim(-.01,.01)
fig.legend()

In [ ]:
#SPECIAL PLOT FOR INCLUDING REGULARIZED RATE MEASUREMENTS

fig, ax = plt.subplots(figsize=(4.75,4.75))
#fig, ax = plt.subplots(1,2, figsize=(8,4), width_ratios=[.8,2])


prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
theo = d[0]['theo']
#theo = rate_theo
rate_theo = theo

C = list(set( [ tuple(i) for i in params[:,:-1]])  )
C.sort()
C = np.array(C)

elist = list(set(params[:,:-2].prod(axis=-1)))
elist.sort()


lbls=[]
for val, param in zip(data, params):
    
    np.random.shuffle(val)
    val = val[:]
    

    
    color_code = param[:-1]
    c = colors[ np.where( (C==color_code).all(axis=-1)) [0][0] ]
    #m, s = np.mean(data)/theo-1, (3*np.std(data)/np.sqrt(len(data)-1))/theo
    m, s = np.mean(val), (3*np.std(val)/np.sqrt(len(val)-1))
    #print(m,s)
    if abs((m-theo)/theo) < 100:

        steps = param[-2]
        lbl = f"oneshot:{list(param[:2])}$\\times${param[2]}"
        #lbl = f"E:{param[:3]} S:{steps}_t:{len(val)}_N{param[-1]/10_000}"
        if lbl in lbls:
            lbl = None

        offset = 1+.025 * ( -(len(elist)-1)/2 + (np.where(param[:-2].prod() == elist)[0][0]))
        ax.errorbar(param[-1]*offset, (m-theo)/theo, marker='o', yerr=s/theo, label=lbl, linestyle='-', color=c)
        
        #ax[1].errorbar(param[-1]*np.random.uniform(.9,1.1), (m-theo)/theo, marker='o', yerr=s/theo, markersize=4,linewidth=.8, color=c)
        #ax.errorbar(param[-1]*offset, (m-theo)/theo, marker='o', yerr=s/theo, markersize=4,linewidth=.8, color=c)

        lbls.append(lbl)

for E, N, EP in zip(rate_E, rate_N, rate_EP):
    temp_N = [np.random.uniform(.95,1.05)*i for i in N]
    #ax.errorbar(temp_N, (EP[:,0]-rate_theo)/rate_theo, yerr=EP[:,1], color=color_dict[E], alpha=1, markersize=4, linewidth=1.5, marker='D', label=f'{E}')
    lbl = f'rate {[E,E]}$\\times${rate_steps+3}'


    #ax.errorbar( temp_N, (EP[:,0]-rate_theo)/rate_theo, yerr=EP[:,1]/rate_theo, color=color_dict[E], alpha=1, markersize=3, linestyle='--', linewidth=2, marker='D', label=lbl)
    #ax[1].errorbar( temp_N, temp_EP[:,0], yerr=temp_EP[:,1], color=color_dict[E], alpha=1, markersize=6, linestyle='--', linewidth=2, marker='D', label=lbl)

    plot_err = [ item[1]/rate_theo if (item[0]-rate_theo)/rate_theo<.05 else 0 for item in EP[:] ]
    ax.errorbar( temp_N[:], (EP[:,0]-rate_theo)/rate_theo, yerr=plot_err, color=color_dict[E], alpha=.7, markersize=6, linestyle='--', linewidth=1, marker='D',label=lbl)

    #ax[0].plot( temp_N[:], (temp_EP[:,0]-rate_theo)/rate_theo, color=color_dict[E], alpha=.5, markersize=4, linestyle='--', linewidth=1, marker='D')





ax.set_yscale('symlog')
ax.set_yticks([-.02,-.01,0, .01,.02])
ax.set_yticklabels([f'{v:.02f}' for v in [-.02,-.01,0, .01,.02]])
ax.set_ylim(-.5,2.5)

#print(d[0]['theo']);

ax.set_xlabel('$N$')
ax.set_ylabel('$\hat{\Sigma}$ relative error')

ax.set_xscale('log')
#ax.set_xscale('log')

#ax[0].set_title('oneshot $\Sigma$ estimation')
#ax[1].set_title('comparison to rate $\Sigma$ estimation')
ax.set_ylim(-.03, .03);
#ax.set_ylim(1.6, 2)
#ax.set_ylim(-80, 50);
#ax.axhline(theo, color='k', label='theo')
#ax.set_xlim(5_000, 550_000)
#ax.set_xlim(5_000,1_500_000)

ax.axhline(0, color='k')
#ax.axhline(0, color='k')

#ax.axhspan(-.05, .05, xmax=.83, facecolor='k', alpha=0.2)
#ax.axhspan(-.05, .05, facecolor='k', alpha=0.2)

fig.legend()
fig.tight_layout()

In [ ]:
fig.savefig('oneshot_reg.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
theo = d[0]['theo']

C = list(set( [ tuple(i) for i in params[:,:-1]])  )
C.sort()
C = np.array(C)

for val, param in zip(data, params):
    color_code = param[:-1]
    c = colors[ np.where( (C==color_code).all(axis=-1)) [0][0] ]
    #m, s = np.mean(data)/theo-1, (3*np.std(data)/np.sqrt(len(data)-1))/theo
    m, s = np.mean(val), (3*np.std(val)/np.sqrt(len(val)-1))
    #print(m,s)
    steps = param[-2]
    lbl = f"E:{param[:3]} S:{steps}_t:{len(val)}"
    ax.errorbar(param[-1]*np.random.uniform(.95,1.05), (m-theo)/theo, marker='o', yerr=s/theo, label=lbl, color=c)


fig.legend()


print(d[0]['theo']);

ax.set_xlabel('num_paths')
ax.set_ylabel('EPR relative error')
ax.set_xscale('log')
ax.set_title('oneshot EPR estimation')
ax.set_ylim(-.1, .1);
#ax.set_ylim(1.6, 2)
#ax.set_ylim(-80, 50);
#ax.axhline(theo, color='k', label='theo')
ax.set_xlim(5_000,350_000)
ax.axhline(0, color='k', label='theo')

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
theo = d[0]['theo']

C = list(set( [ tuple(i) for i in params[:,:-1]])  )
C.sort()
C = np.array(C)

for val, param in zip(data, params):
    color_code = param[:-1]
    c = colors[ np.where( (C==color_code).all(axis=-1)) [0][0] ]
    #m, s = np.mean(data)/theo-1, (3*np.std(data)/np.sqrt(len(data)-1))/theo
    m, s = np.mean(val), (3*np.std(val)/np.sqrt(len(val)-1))
    #print(m,s)
    steps = param[-2]
    lbl = f"E:{param[:3]} S:{steps}_t:{len(val)}"
    ax.errorbar(param[-1]*np.random.uniform(.95,1.05), (m-theo)/theo, marker='o', yerr=s/theo, label=lbl, color=c)


fig.legend()


print(d[0]['theo']);

ax.set_xlabel('num_paths')
ax.set_ylabel('EPR relative error')
ax.set_xscale('log')
ax.set_title('oneshot EPR estimation')
ax.set_ylim(-.1, .1);
#ax.set_ylim(1.6, 2)
#ax.set_ylim(-80, 50);
#ax.axhline(theo, color='k', label='theo')
ax.set_xlim(45_000,400_000)
ax.axhline(0, color='k', label='theo')

In [ ]:
d[-1]['data']

In [ ]:
for dat,p in zip(data, params):
    print(p, len(dat))
    print( np.max(dat), np.min(dat), np.mean(dat), 3*np.std(dat) )

In [ ]:
fig, ax = plt.subplots()
ax.plot(data[-2])
ax.plot(data[-1])
ax.axhline(1.601)

In [ ]:
MU = d[-1]['mus'].mean(axis=0)
XI = d[-1]['Xis'].mean(axis=0)

EP = MU @ np.linalg.inv(XI) @ MU

In [ ]:
XX = d[-3]['Xis'][]

In [ ]:
d[8]['data']

In [ ]:
fig.savefig('oneshot_EP_first_results.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
theo = d2[0]['theo']

E = list(set([ tuple( [*i['num_est'],i['proc_params']['sim'][1]] )  for i in d2]))
E.sort()
E = np.array(E)

for dct in d2:
    data = dct['data']
    color_code = [*dct['num_est'],dct['proc_params']['sim'][1]]
    c = colors[ np.where( (E==color_code).all(axis=-1)) [0][0] ]
    #m, s = np.mean(data)/theo-1, (3*np.std(data)/np.sqrt(len(data)-1))/theo
    m, s = np.mean(data), (3*np.std(data)/np.sqrt(len(data)-1))
    #print(m,s)
    steps = int(dct['proc_params']['sim'][1])
    lbl = f"E:{dct['num_est']} S:{steps}"
    ax.errorbar(dct['params']['sim'][0]*np.random.uniform(.95,1.05), (m-theo)/theo, marker='o', yerr=s/theo, label=lbl, color=c)


fig.legend()


print(d2[0]['theo']);

ax.set_xlabel('num_paths')
ax.set_ylabel('EPR relative error')
ax.set_xscale('log')
ax.set_title('oneshot EPR estimation')
ax.set_ylim(-.2, .2);
#ax.set_ylim(1.6, 2)
#ax.set_ylim(-80, 50);
#ax.axhline(theo, color='k', label='theo')
ax.set_xlim(45_000,350_000)
ax.axhline(0, color='k', label='theo')


